# Naive Bayes
**Clive Chan, 2017**

It's a very simple algorithm, actually. You take a bunch of observed features individually and determine the probability of observing that feature given each of the possible predictions. You then multiply all of those together and see which prediction has the lowest probability. Bayes' theorem applied repeatedly, with an independence assumption.

I'm going to be using a few "prior" classes, which basically streamingly update a model. Each will implement an `update` function, which takes a single observation and updates the internal model, and a `predict` function, which takes a potential observation and returns the probability that it will be observed, udner the current model.

## Models
### Counter
Just plain old counting and bucketing.

In [1]:
from collections import defaultdict
from math import log
class CounterModel:
    def __init__(self):
        self.totalCount = 0
        self.counter = defaultdict(int)
    def update(self, value):
        self.counter[value] += 1
        self.totalCount += 1
    def predict(self, value):
        return self.counter[value] / self.totalCount
    def predict_log(self, value):
        return log(self.counter[value]) - log(self.totalCount)

### Normal
Assumes a normal prior.

#### Proof of online variance algorithm [Welford's Algorithm] used below
Following is based on [this](http://jonisalonen.com/2013/deriving-welfords-method-for-computing-variance/).

We are calculating the sum of squared deviations (M2) in an online fashion, then dividing by the total count to get variance. Let's look at the difference between two consecutive M2s, which are indicated by $(N-1)s_N$ and $(N-2)s_{N-1}$ (since $s_N$ means sample standard deviation):

$$
\begin{align*}
&(N-1)s_N - (N-2)s_{N-1} \\
&= \sum^{N}_{i=1} (x_i - \bar{x}_N)^2 - \sum^{N-1}_{i=1} (x_i - \bar{x}_{N-1})^2 \\
&= (x_N - \bar{x}_N)^2 + \sum^{N-1}_{i=1} (x_i - \bar{x}_N)^2 - (x_i - \bar{x}_{N-1})^2 \\
&= (x_N - \bar{x}_N)^2 + \sum^{N-1}_{i=1} (x_i - \bar{x}_N + x_i - \bar{x}_{N-1})(x_i - \bar{x}_N - x_i + \bar{x}_{N-1}) \\
&= (x_N - \bar{x}_N)^2 + (- \bar{x}_N + \bar{x}_{N-1}) \sum^{N-1}_{i=1} (2x_i - \bar{x}_N - \bar{x}_{N-1}) \\
&= (x_N - \bar{x}_N)^2 + (\bar{x}_{N-1} - \bar{x}_N) \left( 2 \sum^{N-1}_{i=1} x_i - (N-1)\bar{x}_N - (N-1)\bar{x}_{N-1} \right) \\
&= (x_N - \bar{x}_N)^2 + (\bar{x}_{N-1} - \bar{x}_N) \left( (N-1)\bar{x}_{N-1} - N\bar{x}_N + \bar{x}_N \right) \\
&= (x_N - \bar{x}_N)^2 + (\bar{x}_{N-1} - \bar{x}_N) \left( \bar{x}_N - x_N \right) \\
&= (x_N - \bar{x}_N)((x_N - \bar{x}_N) - (\bar{x}_{N-1} - \bar{x}_N)) \\
&= (x_N - \bar{x}_N)(x_N - \bar{x}_{N-1}) \\
\end{align*}
$$

This makes a single pass very easy! See line 10 below.

In [2]:
from math import exp, pi, sqrt

class NormalModel:
    def __init__(self):
        self.n = 0
        self.mean = 0
        self.sumSquaredDeviations = 0
    def update(self, x):
        self.n += 1
        delta = x - self.mean
        self.mean += delta/self.n
        self.sumSquaredDeviations += delta * (x - self.mean)
    def predict(self, value):
        variance = self.sumSquaredDeviations / (self.n - 1)
        exponent = - (value - self.mean)**2 / (2 * variance)
        return 1/sqrt(2 * pi * variance) * exp(exponent)
    def predict_log(self, value):
        variance = self.sumSquaredDeviations / (self.n - 1)
        exponent = - (value - self.mean)**2 / (2 * variance)
        return - 0.5 * log(2 * pi * variance) + exponent

Alright, time for Naive Bayes. I'll follow the same kind of `update`, `predict`, `predict_log` format.

In [3]:
from collections import defaultdict
from functools import reduce
import operator

class NaiveBayes:
    def __init__(self, models):
        self.models = [defaultdict(lambda: model()) for model in models]
        self.outputSet = set()
    def update(self, inputs, output):
        self.outputSet.add(output)
        for x, model in zip(inputs, self.models):
            model[output].update(x)
    def predict(self, inputs):
        # Modify this to actually return the probability?
        return max(((reduce(lambda a, b: a * b,
                   (model[output].predict(x) for x, model in zip(inputs, self.models))
                  ), output) for output in self.outputSet), key=operator.itemgetter(0))[1]
    def predict_log(self, inputs):
        return max(( (sum(model[output].predict_log(x) for x, model in zip(inputs, self.models)), output)
                    for output in self.outputSet), key=operator.itemgetter(0))[1]

Now let's train the model on some random data.

In [4]:
nb = NaiveBayes([NormalModel, CounterModel])
train_data = [
    ([1,"a"], "one"),
    ([6,"b"], "two"),
    ([2,"a"], "one"),
    ([5,"b"], "two"),
    ([3,"a"], "one"),
    ([4,"b"], "two"),
]
for datum in train_data:
    nb.update(datum[0], datum[1])

Time to test!

In [5]:
test_inputs = [
    [-100, "a"],
    [-3, "a"],
    [-2, "a"],
    [-1, "a"],
    [0, "a"],
    [1, "a"],
    [2, "a"],
    [3, "a"],
    [4, "a"],
    [5, "a"],
    [6, "a"],
    [+100, "a"],
]
for input in test_inputs:
    print(nb.predict(input))

one
one
one
one
one
one
one
one
one
one
one
one


The above results with ones and twos are confusing, I think I'm doing something wrong. Possibly:
- Numerical instability with the far ends of the normal distribution?
    - The problem is that using predict_log on counter doesn't make any sense. In fact, using the counter at all makes no sense - it will return a zero probability for anything it hasn't seen yet. Is there anything I can do about this?

... in progress ...

Improvements might include using `predict_log` (returning the log-likelihood rather than the raw likelihood), since that's somewhat more numerically stable for products of probabilities like this. But I haven't yet figured out a nice way of solving the $\ln 0$ problem, since that introduces immense numerical instability.